In [68]:
#### import global modules
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from yaml import safe_load
import google.oauth2.credentials
from google.cloud import bigquery
import gc

# Set global vars
pth_project = Path(os.getcwd().split('notebooks')[0])
pth_data = pth_project / 'data'
pth_queries = pth_project / 'core' / 'queries'
pth_creds = pth_project / 'conf' / 'local' / 'project_config.yaml'
sys.path.insert(0, str(pth_project))
d_project_config = safe_load(pth_creds.open())
# d_params = safe_load((pth_project / 'core' / 'parameters' / 'common.yaml').open())['data_extract']

# import local modules
from gcp import connect_bq_services
# from core.etl.extract import extract_bq_data, extract_pr_codes, format_conv_df, filter_convs

# Connect to google services
bq_client = connect_bq_services(d_project_config['gcp-project-name'])
pd.options.display.max_rows = 100

In [69]:
def extract_bq_data(bq_client, sql=None, pth_query=None):
    if sql is not None:
        df = bq_client.query(sql).to_dataframe()
    elif pth_query is not None:
        sql = pth_query.read_text()
        df = bq_client.query(sql).to_dataframe()
    else:
        raise ValueError('`sql` or `pth_query` should be set')  
    return df


**BAN_Prod_Mix_mapping_data:** BAN level info of product mix

**ADC Inventory Device Data:** Device data for all ADC customers (ADT and Telus) at ADC customer_level

**ADC_BAN_Mapping_data:** Mapping between ADC customerd_id and BAN




In [70]:
Product_Mix_Query='''

SELECT distinct cast( ban as STRING) as BAN, prod_mix FROM `bi-srv-features-pr-ef5a93.ban_product.bq_ban_product_mix` 
where date(part_dt) = '2023-02-01' 
and prod_mix is not null


'''

In [71]:
Product_Mix_DF=extract_bq_data(bq_client, sql=Product_Mix_Query)

In [72]:
Product_Mix_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7815499 entries, 0 to 7815498
Data columns (total 2 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   BAN       object
 1   prod_mix  object
dtypes: object(2)
memory usage: 119.3+ MB


In [63]:
Product_Mix_DF.tail(15)

,BAN,prod_mix
7815484,601949560,"HSIC, OTHER, SING, SMHM, TOS, TTV"
7815485,605502111,SMHM
7815486,605515794,SMHM
7815487,605441269,SMHM
7815488,605465102,SMHM
7815489,605526618,SMHM
7815490,605592696,SMHM
7815491,605668751,SMHM
7815492,605586021,SMHM
7815493,605539670,SMHM


In [36]:
Product_Mix_DF['prod_mix'].value_counts()

C                                                5212410
HSIC                                              344060
HSIC, TTV                                         268657
SING                                              233448
HSIC, OTHER, SING, TTV                            189295
                                                  ...   
SMHM, TOS, WIFIPLUS                                    1
SMHM, WIFIPLUS                                         1
HSIC, SING, SMHM, STV, TTV, WHSIA                      1
SING, STV, TOS, WHSIA, WIFIPLUS                        1
HSIC, OTHER, SING, SMHM, STV, TOS, TTV, WHSIA          1
Name: prod_mix, Length: 262, dtype: int64

In [37]:
BAN_Cust_mapping='''

WITH
  ADC_Customer_Base AS (
  SELECT
    customer_id, ## ADC level
    dealer_customer_id,
    dealer_name,
    join_date,
    CASE
      WHEN account_type_name='Standalone' THEN 'Smart_Camera'
      WHEN account_type_name='Awareness and Automation' THEN 'Smart_Automation_Plus'
    ELSE
    'Monitored'
  END
    AS Package,
    primary_phone
  FROM
    `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_account_details`
  WHERE
    DATE(last_updt_ts) ='2023-02-01'
    AND dealer_name IN ('TELUS Communications Inc.',
      'ADT by TELUS') QUALIFY ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY last_updt_ts DESC) = 1
  ORDER BY
    dealer_name,
    join_date,
    customer_id ),
 
 
 Telus_customers AS (
  SELECT
    bacct_bus_bacct_num AS BAN,
    cust_bus_cust_id,
    pi_prod_instnc_typ_cd,
    pi_prod_instnc_stat_ts,
    prod_instnc_ts,
    pi_cntrct_start_ts AS contract_start_date,
    pi_cntrct_end_ts AS contract_end_date
  FROM
    `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht`
  WHERE
    DATE(prod_instnc_ts) = '2023-02-01'
    AND pi_prod_instnc_typ_cd ='SMHM' #Serice type
    AND bus_prod_instnc_src_id = 1001 #BANs that are FOR home services
    AND pi_prod_instnc_stat_cd IN ('A')
    AND consldt_cust_typ_cd = 'R'
  ORDER BY
    cust_bus_cust_id )



SELECT
  *
FROM
  ADC_Customer_Base AS ADC
LEFT JOIN
  Telus_customers AS Telus
ON
  ADC.dealer_customer_id=Telus.cust_bus_cust_id




'''

In [57]:
BAN_cust_mapping_DF=extract_bq_data(bq_client,sql=BAN_Cust_mapping)

In [58]:
BAN_cust_mapping_DF.head()

,customer_id,dealer_customer_id,dealer_name,join_date,Package,primary_phone,BAN,cust_bus_cust_id,pi_prod_instnc_typ_cd,pi_prod_instnc_stat_ts,prod_instnc_ts,contract_start_date,contract_end_date
0,1665141,105512445,TELUS Communications Inc.,2012-09-18 23:34:41+00:00,Monitored,+19055673949,605671217,105512445,SMHM,2023-01-30 00:00:00,2023-02-01 00:00:00+00:00,2020-11-27 00:00:00,2023-11-27 00:00:00
1,3226085,105215445,TELUS Communications Inc.,2014-11-21 18:03:16+00:00,Monitored,+14165434905,605589459,105215445,SMHM,2022-12-12 00:00:00,2023-02-01 00:00:00+00:00,2020-05-27 00:00:00,2023-05-27 00:00:00
2,3500386,105483913,TELUS Communications Inc.,2015-04-24 10:06:53+00:00,Monitored,+19059701718,605661375,105483913,SMHM,2023-01-24 00:00:00,2023-02-01 00:00:00+00:00,2021-02-22 00:00:00,2024-02-22 00:00:00
3,4335928,99855921,TELUS Communications Inc.,2016-05-16 10:55:13+00:00,Monitored,+14033965798,604048330,99855921,SMHM,2019-10-08 00:00:00,2023-02-01 00:00:00+00:00,2016-10-01 00:00:00,2019-10-01 00:00:00
4,4472908,101519015,TELUS Communications Inc.,2016-06-30 13:35:16+00:00,Monitored,+17782880879,604453551,101519015,SMHM,2020-11-12 00:00:00,2023-02-01 00:00:00+00:00,2017-11-01 00:00:00,2020-11-01 00:00:00


In [65]:
BAN_cust_mapping_DF[BAN_cust_mapping_DF.BAN.isna()==True]

,customer_id,dealer_customer_id,dealer_name,join_date,Package,primary_phone,BAN,cust_bus_cust_id,pi_prod_instnc_typ_cd,pi_prod_instnc_stat_ts,prod_instnc_ts,contract_start_date,contract_end_date
340982,299100,,TELUS Communications Inc.,2009-07-27 16:35:47+00:00,Monitored,+17809167755,<NA>,None,None,None,None,None,None
340983,732052,M0137292,TELUS Communications Inc.,2011-03-24 10:48:43+00:00,Monitored,+14504664793,<NA>,None,None,None,None,None,None
340984,844455,09051180,TELUS Communications Inc.,2011-06-02 10:53:24+00:00,Monitored,+15149033845,<NA>,None,None,None,None,None,None
340985,916163,310418,TELUS Communications Inc.,2011-06-30 21:56:07+00:00,Monitored,+17807852668,<NA>,None,None,None,None,None,None
340986,924472,09030965,TELUS Communications Inc.,2011-07-05 08:47:49+00:00,Monitored,+15148676552,<NA>,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
613896,15336644,,ADT by TELUS,2023-01-29 12:39:28+00:00,Monitored,+17057727035,<NA>,None,None,None,None,None,None
613897,15336919,,ADT by TELUS,2023-01-30 09:27:38+00:00,Monitored,+15148054938,<NA>,None,None,None,None,None,None
613898,15339662,3561006,TELUS Communications Inc.,2023-01-30 12:08:55+00:00,Monitored,7807487975,<NA>,None,None,None,None,None,None
613899,15340973,,ADT by TELUS,2023-01-30 13:58:19+00:00,Monitored,+19053174166,<NA>,None,None,None,None,None,None


In [46]:
ADC_Inventory_data_Query='''

SELECT customer_id,combined_device_type_desc as Device_type, count(distinct device_id) as count_of_device 
FROM `divgpras-pr-579355.ADC_updated.ADC_Inventory_device_data` 
Where date(device_removal_date) is NULL
group by customer_id,Device_type

'''

In [47]:
Device_info=extract_bq_data(bq_client, sql=ADC_Inventory_data_Query)

In [48]:
Device_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4023193 entries, 0 to 4023192
Data columns (total 3 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   customer_id      Int64 
 1   Device_type      object
 2   count_of_device  Int64 
dtypes: Int64(2), object(1)
memory usage: 99.8+ MB


In [49]:
Device_info.head()

,customer_id,Device_type,count_of_device
0,13436884,unknown,1
1,14673692,unknown,1
2,14688712,unknown,1
3,14569202,unknown,1
4,14707511,unknown,1


In [50]:
Device_info['Device_type'].value_counts()

Partition                                         608449
Contact                                           476727
Motion                                            426682
Qolsys Panel Camera                               410017
Camera                                            373322
Local Gateway                                     363470
Smoke/Heat                                        290853
Gps                                               153844
Z-Wave Lock                                       134895
CO                                                102161
Z-Wave Thermostat                                  74128
Glassbreak                                         72104
Z-Wave Light                                       59496
Panel Glass Break                                  59060
Water                                              56540
Garage Door                                        54945
Control                                            51293
Panic Button                   

In [51]:
Device_info_wide=Device_info.pivot_table(index='customer_id', columns='Device_type', values='count_of_device',aggfunc = "sum").reset_index()

In [53]:
rep_chars = ' |\|-|:|/'

Device_info_wide.columns = Device_info_wide.columns.str.replace(rep_chars, '_')

/tmp/ipykernel_1018/4226625745.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  Device_info_wide.columns = Device_info_wide.columns.str.replace(rep_chars, '_')


In [56]:
Device_info_wide.head()

Device_type,customer_id,Audio_Controller,Audio_Module,Audio_Zone,Auxiliary_Sensor,Bluetooth_Device,Bluetooth_Speaker,CO,Camera,Card_Reader,...,Z-Wave_Light,Z-Wave_Lock,Z-Wave_Lock___Power_Meter,Z-Wave_Lock___Z-Wave_Light,Z-Wave_Siren,Z-Wave_Thermostat,Z-Wave_Thermostat___Power_Meter,Z-Wave_Unknown,mPERS,unknown
0,174996,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,210137,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,213860,0,0,0,0,1,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
3,217919,0,0,0,0,0,0,0,0,0,...,2,1,0,0,0,1,0,0,0,0
4,218873,0,0,0,0,0,0,0,7,0,...,3,1,0,0,0,0,0,0,0,0


In [55]:
Device_info_wide.fillna(0,inplace=True)

In [66]:
Device_BAN_mapping=BAN_cust_mapping_DF[['customer_id','BAN','cust_bus_cust_id']].merge(Device_info_wide,on='customer_id',how='left')

In [67]:
Device_BAN_mapping.head()

,customer_id,BAN,cust_bus_cust_id,Audio_Controller,Audio_Module,Audio_Zone,Auxiliary_Sensor,Bluetooth_Device,Bluetooth_Speaker,CO,...,Z-Wave_Light,Z-Wave_Lock,Z-Wave_Lock___Power_Meter,Z-Wave_Lock___Z-Wave_Light,Z-Wave_Siren,Z-Wave_Thermostat,Z-Wave_Thermostat___Power_Meter,Z-Wave_Unknown,mPERS,unknown
0,1665141,605671217,105512445,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3226085,605589459,105215445,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,3500386,605661375,105483913,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4335928,604048330,99855921,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4472908,604453551,101519015,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
